In [ ]:
!pip install tensorflow-gan mtcnn  

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow import keras
import tensorflow.compat.v1 as tf1
import tensorflow_datasets as tfds
# import tensorflow_gan as tfgan
import time
import matplotlib.pyplot as plt
import os
import cv2 as cv

import warnings
warnings.filterwarnings("ignore")

from IPython import display
print(tf.__version__)

2.3.0


In [2]:
# This is the TPU initialization code that has to be at the beginning.
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver() #(tpu='grpc://' + ['COLAB_TPU_os.environADDR'])
    print('Running on TPU ', resolver.master())
except ValueError:
    resolver = None

if resolver:
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
from google.cloud import storage
try:
    from google.colab import auth
    auth.authenticate_user()
    credentials=None

except ModuleNotFoundError:

    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_file( #file location of GCS private key
        'xx')

In [ ]:
client = storage.Client(project='deepfake-research', credentials=credentials)
objects = client.list_blobs('celeba-ds-jh', prefix='celeba_all_preprocessed')
tfrecords = []
for object_ in objects:
    path = str(object_).split(', ')[1]
    gs_path = os.path.join('gs://celeba-ds-jh', path)
    tfrecords.append(gs_path)

In [3]:
params = {'batch_size': 128, 
         'image_dims': (192, 128),
         'noise_dims': 100,
         'ds_size': 202599,
         'start_epoch': 1, #1, #40, #90,  
         'end_epoch': 50,
         'model_number': 8} #39, #89, #150

In [ ]:
@tf.autograph.experimental.do_not_convert
def input_function(params, mode=None):
    batch_size = params['batch_size']
    resized_height, resized_width = params['image_dims'] #s/b (192, 128)
        
#todo -- improve documentation 
    
    
    def preprocess_image(img):
        #decode TFexample record
        features_dictionary = {
            'image': tf.io.FixedLenFeature([], tf.string)
        }
        features = tf.io.parse_single_example(img, features_dictionary)
        decoded_image = tf.io.decode_jpeg(features['image'], 3)

        #add dim at the zero axis Shape will be from (x, y, z) -> (None, x, y, z)
        image_tensor = tf.expand_dims(decoded_image, 0)
        #undo the above line -- this is needed due to TF not allowing a filtered tensor py_function
        image_tensor = tf.gather(image_tensor, 0)

        #convert tensor values to between -1 and 1 (0 to 255 -> -1 to 1)
        image_tensor = (tf.cast(image_tensor, tf.float32) - 127.5) / 127.5

        #randomly mirror images
        image_tensor = tf.image.random_flip_left_right(image_tensor)

        return image_tensor
    


    

    
    image_dataset = (tf.data.TFRecordDataset(filenames = [tfrecords]).
                     cache().
                     map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).
                     repeat())
        
    image_dataset = (image_dataset.batch(batch_size,
                                        drop_remainder=True,)
                                        .prefetch(tf.data.experimental.AUTOTUNE))


    return image_dataset
        



In [ ]:
def generator_input_function(params):
    batch_size = params['batch_size']
    noise_dims = params['noise_dims'] #this can be an arbitrary number
    #ds to generate images

    noise_dataset = tf.random.normal([batch_size, noise_dims])
    return noise_dataset

In [ ]:
def disc_set_of_layers(model_, filters_, kernal, strides_, dropout=0):
    '''
    function to add the following layers to a discriminator model:
    Conv2D, MaxPooling2D, BatchNormalization, LeadyReLU, Dropout

    args:
      model_ : tf.keras.Sequential model (discriminator model)
      filters_: int, number of filters in Conv2D layer
      kernal: int, kernal size in Conv2D layer
      strides_: int, stride size in MaxPooling2D layer
      dropout: float, dropout percentage in Dropout layer, default is 0.0
        must be less than 1.0

    returns:
      model_: tf.keras.Sequential model that is the same as the model_ input plus above 
        layers added
    '''
    model_.add(layers.Conv2D(filters_, (kernal, kernal), padding='same'))
    model_.add(layers.MaxPooling2D(strides_, strides_))
    model_.add(layers.BatchNormalization())
    model_.add(layers.LeakyReLU())
    # model_.add(layers.Dropout(dropout))

    return model_


In [ ]:
def discriminator_model(input_shape=[192, 128, 3]):
    
    #consider creating a singel model that takes two inputs provides two outputs
    
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (5, 5), padding='same',
                                     input_shape=input_shape)) #changed from 225*146
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.LeakyReLU()) #- examples of GANs I have found use LeakyReLU after each COnv2D layer
    model.add(layers.Dropout(0.35)) 
    # model.add(layers.Dense(512)) #can add dense layers if desired



    disc_set_of_layers(model, 64, 5, 2, 0.35)
    

    disc_set_of_layers(model, 128, 5, 2, 0.35)

    disc_set_of_layers(model, 256, 5, 2, 0.35)

    # disc_set_of_layers(model, 256, 5, 2, 0.35)

    # disc_set_of_layers(model, 512, 5, 2, 0.35)
    
    model.add(layers.Flatten())
    model.add(layers.Dense(512))

    model.add(layers.Dense(2, activation='sigmoid')) 

    return model
# discriminator_model().summary()

In [4]:
def gen_set_of_layers(x, filters, kernal, stride, bias, activ=None):
    '''
    function to add set of layers to generator model
    the following layers will be added to model:
        Conv2DTranspose
        BatchNormalization
        LeakyRelu
    
    inputs:
        x: input tensor
        filters (int) number of filters in Conv2DTranspose layer
        kernal (int) kernal size (kernal, kernal) of Conv2DTranspose layer
        stride (int) stride size (strides, strides) of Conv2DTranspose layer
        bias (bool) value to be passed into use_bias arg in Conv2DTranspose layer
        activ (srr) activation function of Conv2DTranspose layer
    
    returns:
        x: above referenced layers added to x input
    '''
    x = layers.Conv2DTranspose(filters,
                              (kernal, kernal),
                              strides=(stride, stride),
                              padding='same',
                              use_bias=bias,
                              activation=activ)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    
    return x

In [7]:
def generator_model(params=params): #178 * 218
    input_shape = params['noise_dims']
#     input_shape = 50
    input_shape = (input_shape,)
    
    input_tensor = layers.Input(shape=input_shape)
    dense_layer = layers.Dense(24576, use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(dense_layer)
    x = layers.LeakyReLU()(x)
    
    #reshape 
    x = layers.Reshape((6, 4, 1024))(x)
    

    x = gen_set_of_layers(x, 1024, 5, 1, False, None) #was 1024, 5, 2

    for _ in range(4):
        num_filters = 512
        x = gen_set_of_layers(x, num_filters, 5, 2, True, None)
        num_filters /= 2
    
#     x = gen_set_of_layers(x, 512, 5, 2, True, None)
    
#     x = gen_set_of_layers(x, 256, 5, 2, True, None)


#     x = gen_set_of_layers(x, 128, 5, 2, True, None)


#     x = gen_set_of_layers(x, 64, 5, 2, True, None)
    
    #number of filters on last layer must be equal to 3 (one for each of R, G, B)
    x = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    output = layers.Activation('tanh')(x)

    assert(output.shape[1:] == ( *params['image_dims'], 3))
    
    model = keras.Model(inputs=input_tensor, outputs=output, name='generator')

    return model
generator_model().summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 24576)             2457600   
_________________________________________________________________
batch_normalization_8 (Batch (None, 24576)             98304     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 24576)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 6, 4, 1024)        0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 6, 4, 1024)        26214400  
_________________________________________________________________
batch_normalization_9 (Batch (None, 6, 4, 1024)        40

In [ ]:
(None, *params['image_dims'], 3)

In [ ]:
def loss_function(real_output, fake_output):
    '''
    takes output from a discriminator GAN model for a batch of real and fake images
    and returns the loss for the discriminator and generator in a GAN model
    
    args:
        real_output: output from a batch of real images passed through a discriminator, a tensor
        shapped (batch_size, 1)
        
        fake_output: output from a batch of fake images generated by a generator GAN model, 
        passed into a discriminator GAN model, a tensor shapped (batch_size, 1)
        
    returns:
        generator_loss: the generator loss for the training batch
        
        discriminator_loss: the discriminator loss for the training batch
    '''

    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.NONE)
    generator_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    #add some noise to target values
    extra_noise = tf.random.uniform((1,), 0, 0.1)
    discriminator_loss_fake = cross_entropy(tf.zeros_like(fake_output) + extra_noise, fake_output)
    discriminator_loss_real = cross_entropy(tf.ones_like(real_output) - extra_noise, real_output)
    discriminator_loss = discriminator_loss_fake + discriminator_loss_real
    
    #keep track of loss values
    gen_loss.update_state(generator_loss)
    disc_loss.update_state(discriminator_loss)
    
    #keep track of accuracy
    acc_fake.update_state(tf.zeros_like(fake_output), fake_output)
    acc_real.update_state(tf.ones_like(real_output), real_output) #these are backwards #fixed
  
    return generator_loss, discriminator_loss

In [ ]:
try:
    model_bucket = client.bucket('jh-gan-testing')
    model_name_generator =  'na' # 'gen6_epoch230.h5'  #'gen_5.h5' 
    model_name_discriminator =  'na'  # 'disc6_epoch230.h5' #'disc_5.h5' 
    model_blob_g = model_bucket.blob(model_name_generator)
    model_blob_g.download_to_filename(model_name_generator)
    model_blob_d = model_bucket.blob(model_name_discriminator)
    model_blob_d.download_to_filename(model_name_discriminator)
    # create the models and optimizers in the strategy.scope
    with strategy.scope():
        generator = tf.keras.models.load_model(model_name_generator)
        discriminator = tf.keras.models.load_model(model_name_discriminator)
    print('loaded partially trained models')
except:
    with strategy.scope():
        generator = generator_model()
        discriminator = discriminator_model()
    print('created new models')


with strategy.scope():
    generator_optimizer = tf.keras.optimizers.Adam(1e-4) 
    discriminator_optimizer = tf.keras.optimizers.Adam(5e-5) #SGD #Adam
    #metrics
    #loss
    gen_loss = tf.keras.metrics.Mean('gen_loss', dtype=tf.float32)
    disc_loss = tf.keras.metrics.Mean('disc_loss', dtype=tf.float32)

    #discriminator accuracy on real and fake images
    acc_real = tf.keras.metrics.BinaryAccuracy('acc_real', threshold=0.5)
    acc_fake = tf.keras.metrics.BinaryAccuracy('acc_fake', threshold=0.5)

In [ ]:
@tf.function
def training_step(params, real_images_, train_disc=False, train_gen=False):
    generator_input_ = generator_input_function(params)

#make the below a function
#inputs real_images, generator_input
    def disc_train_step(generator_input, real_images):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_images = generator(generator_input, training=False)


            real_output = discriminator(real_images, training=True)
            fake_output = discriminator(fake_images, training=True)

            generator_loss, discriminator_loss = loss_function(real_output, fake_output)

        #update disc gradients
        discriminator_gradients = disc_tape.gradient(discriminator_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

    def gen_train_step(generator_input, real_images):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_images = generator(generator_input, training=True)


            real_output = discriminator(real_images, training=False)
            fake_output = discriminator(fake_images, training=False)

            generator_loss, discriminator_loss = loss_function(real_output, fake_output)


        #update generator gradients
        generator_gradients = gen_tape.gradient(generator_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
        


    #will train the disc or generator based on args passed into fn 
    if train_disc:
        strategy.run(disc_train_step, args=(generator_input_, real_images_))
    if train_gen:
        strategy.run(gen_train_step, args=(generator_input_, real_images_))

In [ ]:
def display_generated_pic(gen_output_tensor, return_=False):
    '''
    Function to convert tensor from a range of -1 to 1 -> 0 to 1 and display the resulting image
    Will display the converted tensor as an image and can return a tensor that is converted as per above
    '''
    generated_image = (gen_output_tensor + 1) / 2
    # generated_image = gen_output_tensor
    plt.imshow(generated_image)
    plt.show()
    if return_:
        return generated_image

In [ ]:
ds_size = params['ds_size'] #celeba DS is 162,770 images for training
start_epoch = params['start_epoch']
end_epoch = params['end_epoch']
batch_size = params['batch_size']
params_ = {}
params_['noise_dims'] = params['noise_dims']

params_['batch_size'] = 1
steps_per_epoch = int(tf.math.ceil(ds_size / batch_size))

In [ ]:
real_images = input_function(params)

In [ ]:
#training code
step_counter = 0
model_num = params['model_number']
loss_gen = []
loss_disc = []
real_accuracy = []
fake_accuracy = []
step = []
# epochs = end_epoch - start_epoch 
model_bucket = client.bucket('jh-gan-testing')
for epoch in range(start_epoch, end_epoch + 1):
    if epoch % 10 == 0:
        gen_model_name = 'gen{}_epoch{}.h5'.format(model_num, epoch)
        disc_model_name = 'disc{}_epoch{}.h5'.format(model_num, epoch)
        generator.save(gen_model_name)
        discriminator.save(disc_model_name)
        #upload generator model
        blob = model_bucket.blob(gen_model_name)
        blob.upload_from_filename(gen_model_name)

        #upload disc model
        blob = model_bucket.blob(disc_model_name)
        blob.upload_from_filename(disc_model_name)


    tf.random.set_seed(epoch)
    if epoch %2 == 1:
        #alternate training discriminator/generator every other epoch
        disc_train, gen_train = True, False
    else:
        disc_train, gen_train = False, True
    tme_start = time.time()
    for image_batch in real_images:
      # start = time.time()
        training_step(params, image_batch, disc_train, gen_train)
        end = time.time()
        tme= end - tme_start
        step_counter +=1
        if step_counter %395 == 394:
            step.append((epoch - 1) * steps_per_epoch + step_counter)
            loss_gen.append(gen_loss.result().numpy())
            loss_disc.append(disc_loss.result().numpy())
            real_accuracy.append(acc_real.result().numpy())
            fake_accuracy.append(acc_fake.result().numpy())

            #reset states of metrics
            gen_loss.reset_states()
            disc_loss.reset_states()
            acc_real.reset_states()
            acc_fake.reset_states()



            

            display.clear_output(wait=True)
            print('step {}, epoch {}'.format(step_counter, epoch))
            print('total time: {} seconds // {} per step'. format ((end - tme_start), 
                                                                   ((end- tme_start) / step_counter)))
            gen_input = generator_input_function(params_)
            gen_output = generator(gen_input)
            _ = display_generated_pic(gen_output[0])
            print(params)
                    #step  #loss
            plt.plot(step, loss_gen, label='gen')
            plt.plot(step, loss_disc, label='disc')
            plt.ylabel('loss')
            plt.xlabel('step')
            plt.legend()#('upper left')
            plt.show()

            #todo - make into a function 
            plt.plot(step, real_accuracy, label='real')
            plt.plot(step, fake_accuracy, label='fake')
            plt.ylabel('accuracy')
            plt.xlabel('step')
            plt.legend()#('upper left')
            plt.show()

        if step_counter > steps_per_epoch:
            step_counter = 0
            #reset states of metrics
            gen_loss.reset_states()
            disc_loss.reset_states()
            acc_real.reset_states()
            acc_fake.reset_states()
            break #end epoch

    

In [ ]:
gen_model_path = 'gen_8.h5'
disc_model_path = 'disc_8.h5'

In [ ]:
generator.save(gen_model_path)
discriminator.save(disc_model_path)
blob = model_bucket.blob(gen_model_path)
blob.upload_from_filename(gen_model_path)
blob = model_bucket.blob(disc_model_path)
blob.upload_from_filename(disc_model_path)

In [ ]:
gen_input = generator_input_function(params)
gen_output = generator(gen_input)
_ = display_generated_pic(gen_output[0])

In [ ]:
for x in np.arange(1, 50):
  _ = display_generated_pic(gen_output[x])
  time.sleep(0.4)


In [ ]:
train_data_dictionary = {
    'step': step_counter,
    'gen_loss': loss_gen,
    'disc_loss': loss_disc,
    'real_acc': real_accuracy, 
    'fake_acc': fake_accuracy
}

In [ ]:
train_data_df = pd.DataFrame(train_data_dictionary)
# train_data_df.to_csv('train_data.csv')
train_data_df

In [ ]:
tf.keras.activations.sigmoid(-0.1)